In [1]:
# sbi example taken from ltu-ili examples - understanding ili 
# simpler example to play with parameters/training input/arguments

In [2]:
import os
import pandas as pd
import argparse
import numpy as np
import torch
from torch.distributions import Uniform, ExpTransform, TransformedDistribution #, AffineTransform

import torch.nn as nn

import ili
from ili.dataloaders import StaticNumpyLoader
from ili.inference import InferenceRunner
from ili.validation import ValidationRunner
from ili.dataloaders import NumpyLoader
from ili.inference import InferenceRunner
from ili.validation.metrics import PosteriorCoverage, PlotSinglePosterior

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Device:', device)

Device: cpu


In [3]:
# Get theta
df_pars = pd.read_csv('/home/jovyan/camels/LH/CosmoAstroSeed_IllustrisTNG_L25n256_LH.txt', delim_whitespace=True)
theta = df_pars[['Omega_m', 'sigma_8', 'A_SN1', 'A_AGN1', 'A_SN2', 'A_AGN2']].to_numpy()
print(theta)
print(theta.shape)

[[0.309   0.979   3.11234 1.12194 0.6685  0.53182]
 [0.3026  0.9394  3.42001 3.96137 1.03311 1.1607 ]
 [0.4282  0.753   0.70613 0.37423 1.96292 0.6272 ]
 ...
 [0.1582  0.7854  0.93952 2.23148 1.85446 1.32961]
 [0.3854  0.9778  0.93692 0.42869 1.48761 1.32042]
 [0.4322  0.6142  1.28521 0.35799 1.1591  0.86754]]
(1000, 6)


/tmp/ipykernel_1882/4189281711.py:2: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df_pars = pd.read_csv('/home/jovyan/camels/LH/CosmoAstroSeed_IllustrisTNG_L25n256_LH.txt', delim_whitespace=True)


In [4]:
def initialise_priors(device="cpu", astro=True, dust=True):

    combined_priors = []

    if astro:
        base_dist1 = Uniform(
            torch.log(torch.tensor([0.25], device=device)),
            torch.log(torch.tensor([4], device=device)),
        )
        base_dist2 = Uniform(
            torch.log(torch.tensor([0.5], device=device)),
            torch.log(torch.tensor([2], device=device)),
        )
        astro_prior1 = TransformedDistribution(base_dist1, ExpTransform())
        astro_prior2 = TransformedDistribution(base_dist2, ExpTransform())
        omega_prior = Uniform(
            torch.tensor([0.1], device=device),
            torch.tensor([0.5], device=device),
        )
        sigma8_prior = Uniform(
            torch.tensor([0.6], device=device),
            torch.tensor([1.0], device=device),
        )
        combined_priors += [
            omega_prior,# prior for omega m, between 0.1 and 0.5: uniform
            sigma8_prior,# prior for sigma_8, between 0.6 and 1: uniform
            astro_prior1,# prior for A_SN1, between 0.25 and 4: exponential
            astro_prior1,# prior for A_AGN1, between 0.25 and 4: exponential
            astro_prior2,# prior for A_SN2, between 0.5 and 2: exponential
            astro_prior2,# prior for A_AGN2, between 0.5 and 2: exponential
        ]

    prior = process_prior(combined_priors)

    return prior[0]

In [5]:
from torch.distributions import Uniform, ExpTransform, TransformedDistribution #, AffineTransform
import torch
from sbi.utils.user_input_checks import process_prior

prior = initialise_priors()
print(prior)

MultipleIndependent()


/srv/conda/envs/notebook/lib/python3.9/site-packages/sbi/utils/user_input_checks.py:76: UserWarning: Prior was provided as a sequence of 6 priors. They will be
            interpreted as independent of each other and matched in order to the
            components of the parameter.
  warnings.warn(
/srv/conda/envs/notebook/lib/python3.9/site-packages/sbi/utils/user_input_checks.py:209: UserWarning: Casting 1D Uniform prior to BoxUniform to match sbi batch requirements.
  warnings.warn(


In [6]:
# get x

# Define the directory containing the LH_X files
directory = "/home/jovyan/camels/LH/get_LF/output/"

# Get all files in the directory
files = os.listdir(directory)

# Filter out files that start with "LH_" and end with ".txt"
LH_X_files = [file for file in files if file.startswith("LH_") and file.endswith(".txt")]

# Initialize lists to store data
phia = []
phi_sigmaa = []
binsa = []
LH_X_values = []

# Iterate over LH_X files
for LH_X_file in LH_X_files:
    # Define the file path
    file_path = os.path.join(directory, LH_X_file)
    
    # Extract LH_X value from the file name (remove the ".txt" extension)
    LH_X = LH_X_file[:-4]
    
    # Initialize an empty dictionary to store variable names and their values
    variable_data = {}

    # Open the text file for reading
    with open(file_path, 'r') as file:
        # Initialize variables to store the current variable name and its values
        current_variable_name = None
        current_variable_values = []

        # Iterate over each line in the file
        for line in file:
            # Remove leading and trailing whitespace from the line
            line = line.strip()

            # Check if the line is empty
            if not line:
                continue

            # Check if the line is a variable name
            if line in ['phi', 'phi_sigma', 'hist', 'massBinLimits']:
                # If it's a new variable name, update the current variable name and reset the values list
                if current_variable_name is not None:
                    variable_data[current_variable_name] = current_variable_values
                    current_variable_values = []

                current_variable_name = line
            else:
                # If it's not a variable name, convert the value to float and append it to the values list
                current_variable_values.append(float(line))

        # Add the last variable data to the dictionary
        if current_variable_name is not None:
            variable_data[current_variable_name] = current_variable_values
        
        # Extract specific variables
        phi = variable_data.get('phi')
        phi_sigma = variable_data.get('phi_sigma')
        bins = variable_data.get('massBinLimits')

        phia.append(phi)
        phi_sigmaa.append(phi_sigma)
        binsa.append(bins)
        LH_X_values.append(LH_X)

# Create a DataFrame from the lists
df_x = pd.DataFrame({'LH_X': LH_X_values, 'phi': phia, 'phi_sigma': phi_sigmaa, 'bins': binsa})

# convert pandas series to np.array to get x
x = np.array(df_x['phi'].tolist())
x.shape # shape 11 because they are in 11 bins

(1000, 11)

In [7]:
import argparse
import sys

parser = argparse.ArgumentParser(
    description="Run SBI inference for toy data.")

parser.add_argument(
    "--model", type=str,
    default="NPE",
    help="Configuration file to use for model training.")

# Filter out unrecognized arguments to avoid errors in Jupyter
args, unknown = parser.parse_known_args()

print(args.model)


NPE


In [8]:
# construct a working directory
if not os.path.isdir("toy"):
    os.mkdir("toy")

# push the as numpy files
np.save("toy/theta.npy", theta)
np.save("toy/x.npy", x)


In [9]:
# make a dataloader
from ili.dataloaders import NumpyLoader

loader = NumpyLoader(x=x, theta=theta)

In [10]:
# instantiate your neural networks to be used as an ensemble
nets = [
    ili.utils.load_nde_sbi(engine='NPE', model='maf', hidden_features=50, num_transforms=5),
    ili.utils.load_nde_sbi(engine='NPE', model='mdn', hidden_features=50, num_components=6)
]

# define training arguments
train_args = {
    'training_batch_size': 32,
    'learning_rate': 1e-4
}

# initialize the trainer
runner = InferenceRunner.load(
    backend='sbi',
    engine='NPE',
    prior=prior,
    nets=nets,
    device=device,
    embedding_net=None,
    train_args=train_args,
    proposal=None,
    out_dir=None
)

# run the trainer
runner(loader=loader)


INFO:root:MODEL INFERENCE CLASS: NPE
INFO:root:Training model 1 / 2.


 Neural network successfully converged after 88 epochs.

INFO:root:Training model 2 / 2.


 Training neural network. Epochs trained: 76

/srv/conda/envs/notebook/lib/python3.9/site-packages/sbi/utils/posterior_ensemble.py:142: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self._weights = torch.tensor(weights) / sum(weights)
INFO:root:It took 6223.247726917267 seconds to train models.


 Neural network successfully converged after 77 epochs.

(NeuralPosteriorEnsemble sampler for potential_fn=<EnsemblePotential>,
 [{'epochs_trained': [0, 88],
   'best_validation_log_prob': [-inf, -2.4881199995676675],
   'validation_log_probs': [-10.248655637105307,
    -9.759519577026367,
    -9.231688181559244,
    -8.633052508036295,
    -7.9463300704956055,
    -7.1629438400268555,
    -6.390581448872884,
    -5.643112659454346,
    -4.986670811971028,
    -4.350228627522786,
    -3.921410878499349,
    -3.5881384213765464,
    -3.3920982678731284,
    -3.2772864500681558,
    -3.0944395065307617,
    -3.1019970575968423,
    -3.053021272023519,
    -2.928750197092692,
    -2.9388650258382163,
    -2.88685933748881,
    -2.8387691179911294,
    -2.8021767139434814,
    -2.7712305386861167,
    -2.767303784688314,
    -2.687042156855265,
    -2.6789416472117105,
    -2.637788454691569,
    -2.6699841817220054,
    -2.564267635345459,
    -2.632283369700114,
    -2.5773824055989585,
    -2.6316495736440024,
    -2.6018803119659424,
    -2.

In [11]:
# use the trained posterior model to predict on a single example from
# the test set
val_runner = ValidationRunner.from_config(
    f"configs/val/toy_sbi_{args.model}.yaml")

val_runner(loader=all_loader)

FileNotFoundError: [Errno 2] No such file or directory: 'configs/val/toy_sbi_NPE.yaml'